In [24]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import pylab as pl
import itertools
import warnings
warnings.filterwarnings('ignore')

from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score

print('All libraries have been imported')

All libraries have been imported


In [25]:
address1 = 'Philadelphia, Pennsylvania'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))

address2 = 'New York, New York'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'Boston, Massachusetts'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))

The geograpical coordinate of Philadelphia, Pennsylvania are 39.9527237, -75.1635262.
The geograpical coordinate of New York, New York are 40.7127281, -74.0060152.
The geograpical coordinate of Boston, Massachusetts are 42.3602534, -71.0582912.


In [26]:
CLIENT_ID = 'XYZOPXMHE50CTZZQYT4U3ZXMJFTDADCUBKEUG5XMKQ3TEQDY' 
CLIENT_SECRET = '0JYVUC2BHPOQVBAMHUBIT2BDK04KKMVWC4CTEDYJZFLD1FIC' 
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 
radius = 1000 

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)

# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

print(url1, url2, url3)

Your credentails:
CLIENT_ID: XYZOPXMHE50CTZZQYT4U3ZXMJFTDADCUBKEUG5XMKQ3TEQDY
CLIENT_SECRET:0JYVUC2BHPOQVBAMHUBIT2BDK04KKMVWC4CTEDYJZFLD1FIC
https://api.foursquare.com/v2/venues/explore?&client_id=XYZOPXMHE50CTZZQYT4U3ZXMJFTDADCUBKEUG5XMKQ3TEQDY&client_secret=0JYVUC2BHPOQVBAMHUBIT2BDK04KKMVWC4CTEDYJZFLD1FIC&v=20180604&ll=39.9527237,-75.1635262&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=XYZOPXMHE50CTZZQYT4U3ZXMJFTDADCUBKEUG5XMKQ3TEQDY&client_secret=0JYVUC2BHPOQVBAMHUBIT2BDK04KKMVWC4CTEDYJZFLD1FIC&v=20180604&ll=40.7127281,-74.0060152&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=XYZOPXMHE50CTZZQYT4U3ZXMJFTDADCUBKEUG5XMKQ3TEQDY&client_secret=0JYVUC2BHPOQVBAMHUBIT2BDK04KKMVWC4CTEDYJZFLD1FIC&v=20180604&ll=42.3602534,-71.0582912&radius=1000&limit=100


In [27]:
# Scrape data

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# Enter the function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# Venues in Philadelphia

venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = pd.json_normalize(venues1) # flatten JSON

# Filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 = nearby_venues1.loc[:, filtered_columns1]

# Filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# Clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

# Venues in New York

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = pd.json_normalize(venues2) # flatten JSON

# Filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 = nearby_venues2.loc[:, filtered_columns2]

# Filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# Clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

# Venues in Boston

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = pd.json_normalize(venues3) # flatten JSON

# Filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 = nearby_venues3.loc[:, filtered_columns3]

# Filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# Clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]





print('{} venues for Philadelphia, PA were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print()
print('{} venues for New York, NY were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print()
print('{} venues for Boston, MA were returned by Foursquare.'.format(nearby_venues3.shape[0]))

100 venues for Philadelphia, PA were returned by Foursquare.

100 venues for New York, NY were returned by Foursquare.

100 venues for Boston, MA were returned by Foursquare.


In [28]:
# Add locations data

nearby_venues1['city'] = 'Philadelphia'
nearby_venues2['city'] = 'New York'
nearby_venues3['city'] = 'Boston'

In [29]:
# Combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)

In [18]:
nearby_venues

,name,categories,lat,lng,id,city
0,Dilworth Park,Park,39.952772,-75.164723,4bde0d566198c9b6c5cc12ff,Philadelphia
1,La Colombe Coffee Roasters,Coffee Shop,39.951659,-75.165238,4de4e118b0fb267148be318b,Philadelphia
2,The Wanamaker Organ,Concert Hall,39.951536,-75.162752,4c5cc27a6ebe2d7f7d2cd12e,Philadelphia
3,City Hall Courtyard,Plaza,39.952484,-75.163592,4f29e8e3e4b02f0aff55b2a7,Philadelphia
4,Fogo De Chão,Churrascaria,39.950917,-75.162971,4a4268fdf964a520d4a51fe3,Philadelphia
5,"The Ritz-Carlton, Philadelphia",Hotel,39.951446,-75.164149,4a68db6ff964a52023cb1fe3,Philadelphia
6,Philadelphia Film Center,Movie Theater,39.950835,-75.164683,47bdd66df964a520da4d1fe3,Philadelphia
7,Blick Art Materials,Arts & Crafts Store,39.950621,-75.163159,4ac8cea6f964a520a8bc20e3,Philadelphia
8,JFK Plaza / Love Park,Plaza,39.954123,-75.165303,4aa14edef964a5206d4020e3,Philadelphia
9,Trader Joe's,Grocery Store,39.954191,-75.162434,5db9d69637706f0008294075,Philadelphia


In [30]:
nearby_venues['categories'].unique()

array(['Park', 'Coffee Shop', 'Concert Hall', 'Plaza', 'Churrascaria',
       'Hotel', 'Movie Theater', 'Arts & Crafts Store', 'Grocery Store',
       'Steakhouse', 'Art Museum', 'American Restaurant', 'Garden',
       'Skating Rink', 'Mediterranean Restaurant',
       'Vegetarian / Vegan Restaurant', 'Market', 'Seafood Restaurant',
       'Wine Bar', 'Bar', 'Mexican Restaurant', 'Donut Shop', 'Bakery',
       'Pizza Place', 'Sandwich Place', 'Taco Place', 'Snack Place',
       'Asian Restaurant', 'Clothing Store', 'Pub', 'Yoga Studio',
       'Deli / Bodega', 'Scenic Lookout', 'Dessert Shop',
       'New American Restaurant', 'Gym', 'Optical Shop',
       'Thai Restaurant', 'Spa', 'Israeli Restaurant', 'Organic Grocery',
       'Restaurant', 'Smoke Shop', 'Falafel Restaurant',
       'Italian Restaurant', 'Discount Store', 'Salad Place',
       'Food Truck', 'Chinese Restaurant', 'Indian Restaurant',
       'Boutique', 'Gift Shop', 'Korean Restaurant', 'Gastropub',
       'Sushi Resta

In [31]:
# Remove all non-restaurant data

nearby_venues['categories'].unique()

removal_list =  ['Park', 'Concert Hall', 'Plaza', 
                 'Hotel', 'Movie Theater', 'Arts & Crafts Store', 'Grocery Store',
                 'Art Museum', 'Garden',
                 'Skating Rink',
                 'Market',
                 'Wine Bar', 'Bar',
                 'Clothing Store', 'Pub', 'Yoga Studio',
                 'Scenic Lookout', 
                 'Gym', 'Optical Shop',
                 'Spa', 'Organic Grocery',
                 'Smoke Shop', 
                 'Discount Store', 
                 'Boutique', 'Gift Shop', 
                 'General Entertainment', 
                 'Cycle Studio', 
                 'Lingerie Store', 'Cosmetics Shop', 'Opera House', 'Hotel Bar',
                 'Laundry Service', 'Dance Studio', 'Gym / Fitness Center',
                 'Boxing Gym', 'Monument / Landmark',
                 'Furniture / Home Store', 'Antique Shop',
                 'Nail Salon', 'Martial Arts School', 
                 'Electronics Store', 'Comic Shop', 'Shopping Mall',
                 'Auditorium', 'Memorial Site', 
                 'Playground', 'Paper / Office Supplies Store', 'Historic Site',
                 'Bookstore', 'Museum',
                 'Kids Store', 'Art Gallery', 'Lounge',
                 'Speakeasy', 'Circus', 
                 'Library', 'Comedy Club', 'Neighborhood', 'Ski Chalet',
                 'Salon / Barbershop', 'Trail',
                 'History Museum']

nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique().tolist()

['Coffee Shop',
 'Churrascaria',
 'Steakhouse',
 'American Restaurant',
 'Mediterranean Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Seafood Restaurant',
 'Mexican Restaurant',
 'Donut Shop',
 'Bakery',
 'Pizza Place',
 'Sandwich Place',
 'Taco Place',
 'Snack Place',
 'Asian Restaurant',
 'Deli / Bodega',
 'Dessert Shop',
 'New American Restaurant',
 'Thai Restaurant',
 'Israeli Restaurant',
 'Restaurant',
 'Falafel Restaurant',
 'Italian Restaurant',
 'Salad Place',
 'Food Truck',
 'Chinese Restaurant',
 'Indian Restaurant',
 'Korean Restaurant',
 'Gastropub',
 'Sushi Restaurant',
 'Hot Dog Joint',
 'Cuban Restaurant',
 'Gourmet Shop',
 'Comfort Food Restaurant',
 'Shanghai Restaurant',
 'Breakfast Spot',
 'French Restaurant',
 'Burger Joint',
 'Juice Bar',
 'Molecular Gastronomy Restaurant',
 'Café',
 'Wine Shop',
 'Cocktail Bar',
 'Bagel Shop',
 'Fried Chicken Joint',
 'Farmers Market',
 'Australian Restaurant',
 'Ice Cream Shop',
 'Belgian Restaurant',
 'Greek Restaurant',
 'B

In [32]:
# Pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[395, 335, 275, 130, 391, 26, 319, 57, 676, 155, 45, 44, 408, 101, 66, 340, 61, 153, 131, 20, 116, 175, 19, 674, 189, 63, 318, 610, 57, 244, 17, 21, 131, 32, 146, 42, 177, 215, 140, 72, 92, 63, 322, 215, 123, 167, 429, 191, 18, 42, 420, 171, 330, 10, 8, 88, 156, 664, 85, 295, 79, 120, 502, 406, 130, 183, 52, 318, 90, 86, 27, 173, 466, 330, 85, 379, 10, 21, 70, 408, 534, 722, 59, 1182, 310, 420, 5, 60, 1206, 35, 214, 131, 469, 744, 28, 440, 361, 78, 55, 435, 158, 156, 569, 1140, 296, 25, 59, 367, 20, 36, 22, 157, 46, 123, 57, 48, 226, 647, 85, 136, 80, 27, 84, 106, 20, 15, 659, 54, 104, 402, 118, 73, 55, 45, 194, 282, 495, 28, 32, 13, 285, 129, 287, 343, 82, 433, 27, 498, 85, 5, 135, 56, 15, 118, 14, 91, 102, 1503, 87, 94, 0, 18, 224, 126, 46, 29, 125, 51, 91, 45, 17]


,name,categories,lat,lng,id,city,likes
1,La Colombe Coffee Roasters,Coffee Shop,39.951659,-75.165238,4de4e118b0fb267148be318b,Philadelphia,395
4,Fogo De Chão,Churrascaria,39.950917,-75.162971,4a4268fdf964a520d4a51fe3,Philadelphia,335
10,Del Frisco's Double Eagle Steak House,Steakhouse,39.950956,-75.165459,4ab2ac0bf964a520d66b20e3,Philadelphia,275
13,The Capital Grille,American Restaurant,39.950716,-75.163942,4af2d4cef964a520a9e821e3,Philadelphia,130
16,Barbuzzo Mediterranean Kitchen & Bar,Mediterranean Restaurant,39.949983,-75.161987,4c7ed709a7958cfa30048f2b,Philadelphia,391


In [33]:
nearby_venues.count()

name          171
categories    171
lat           171
lng           171
id            171
city          171
likes         171
dtype: int64

In [35]:
# Generate the raw dataset

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
1,La Colombe Coffee Roasters,Coffee Shop,39.951659,-75.165238,4de4e118b0fb267148be318b,Philadelphia,395
4,Fogo De Chão,Churrascaria,39.950917,-75.162971,4a4268fdf964a520d4a51fe3,Philadelphia,335
10,Del Frisco's Double Eagle Steak House,Steakhouse,39.950956,-75.165459,4ab2ac0bf964a520d66b20e3,Philadelphia,275
13,The Capital Grille,American Restaurant,39.950716,-75.163942,4af2d4cef964a520a9e821e3,Philadelphia,130
16,Barbuzzo Mediterranean Kitchen & Bar,Mediterranean Restaurant,39.949983,-75.161987,4c7ed709a7958cfa30048f2b,Philadelphia,391


In [36]:
# Explore the dataset for the different cuisines

raw_dataset['categories'].unique().tolist()

['Coffee Shop',
 'Churrascaria',
 'Steakhouse',
 'American Restaurant',
 'Mediterranean Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Seafood Restaurant',
 'Mexican Restaurant',
 'Donut Shop',
 'Bakery',
 'Pizza Place',
 'Sandwich Place',
 'Taco Place',
 'Snack Place',
 'Asian Restaurant',
 'Deli / Bodega',
 'Dessert Shop',
 'New American Restaurant',
 'Thai Restaurant',
 'Israeli Restaurant',
 'Restaurant',
 'Falafel Restaurant',
 'Italian Restaurant',
 'Salad Place',
 'Food Truck',
 'Chinese Restaurant',
 'Indian Restaurant',
 'Korean Restaurant',
 'Gastropub',
 'Sushi Restaurant',
 'Hot Dog Joint',
 'Cuban Restaurant',
 'Gourmet Shop',
 'Comfort Food Restaurant',
 'Shanghai Restaurant',
 'Breakfast Spot',
 'French Restaurant',
 'Burger Joint',
 'Juice Bar',
 'Molecular Gastronomy Restaurant',
 'Café',
 'Wine Shop',
 'Cocktail Bar',
 'Bagel Shop',
 'Fried Chicken Joint',
 'Farmers Market',
 'Australian Restaurant',
 'Ice Cream Shop',
 'Belgian Restaurant',
 'Greek Restaurant',
 'B

In [37]:
# Group those cuisines together in categories

european = ['Mediterranean Restaurant', 'Belgian Restaurant', 'Pizza Place',
       'French Restaurant', 'Falafel Restaurant', 'Italian Restaurant',
       'Irish Pub', 'Greek Restaurant', 'French Restaurant']

latin = ['Mexican Restaurant', 'Taco Place', 'Brazilian Restaurant', 
          'Churrascaria', 'Cuban Restaurant']

asian = ['Thai Restaurant', 'Shanghai Restaurant', 'Chinese Restaurant',
         'Indian Restaurant', 'Korean Restaurant',
         'Thai Restaurant', 'Asian Restaurant', 'Sushi Restaurant']

american = ['Vegetarian / Vegan Restaurant', 'Seafood Restaurant',
           'Burger Joint', 'American Restaurant', 'New American Restaurant',
           'Beer Garden']

casual = ['Bagel Shop', 'Sandwich Place', 'Fried Chicken Joint', 
          'Breakfast Spot', 'Food Truck',
          'Deli / Bodega', 'Snack Place']

def conditions(s):
    if s['categories'] in european:
        return 'european'
    if s['categories'] in latin:
        return 'latin'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in casual:
        return 'casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified
1,La Colombe Coffee Roasters,Coffee Shop,39.951659,-75.165238,4de4e118b0fb267148be318b,Philadelphia,395,None
4,Fogo De Chão,Churrascaria,39.950917,-75.162971,4a4268fdf964a520d4a51fe3,Philadelphia,335,latin
10,Del Frisco's Double Eagle Steak House,Steakhouse,39.950956,-75.165459,4ab2ac0bf964a520d66b20e3,Philadelphia,275,None
13,The Capital Grille,American Restaurant,39.950716,-75.163942,4af2d4cef964a520a9e821e3,Philadelphia,130,american
16,Barbuzzo Mediterranean Kitchen & Bar,Mediterranean Restaurant,39.949983,-75.161987,4c7ed709a7958cfa30048f2b,Philadelphia,391,european
17,HipCityVeg,Vegetarian / Vegan Restaurant,39.950343,-75.164046,5852d6aad21e030e68cb5512,Philadelphia,26,american
19,Oyster House,Seafood Restaurant,39.950415,-75.166533,4a281e64f964a520f4941fe3,Philadelphia,319,american
22,Tredici Enoteca,Mediterranean Restaurant,39.949822,-75.162034,565f615b498eeef346958da1,Philadelphia,57,european
23,El Vez,Mexican Restaurant,39.949755,-75.161903,40fdb200f964a520380b1fe3,Philadelphia,676,latin
24,Beiler's Doughnuts,Donut Shop,39.953697,-75.159307,520f8d6b11d2d0dc2971ac57,Philadelphia,155,None


In [38]:
# Make sure the categories have been created correctly

pd.crosstab(index = raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,25
asian,15
casual,18
european,39
latin,8


In [39]:
raw_dataset['likes'].mean()

206.53801169590642

In [41]:
# Create a bin function

def rankings(df):
    
    if df['likes'] <= 60:
        return 3
    
    elif df['likes'] <= 100:
        return 2
    
    elif df['likes'] > 100:
        return 1

In [42]:
# Apply rankings to the dataset

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
1,La Colombe Coffee Roasters,Coffee Shop,39.951659,-75.165238,4de4e118b0fb267148be318b,Philadelphia,395,None,1
4,Fogo De Chão,Churrascaria,39.950917,-75.162971,4a4268fdf964a520d4a51fe3,Philadelphia,335,latin,1
10,Del Frisco's Double Eagle Steak House,Steakhouse,39.950956,-75.165459,4ab2ac0bf964a520d66b20e3,Philadelphia,275,None,1
13,The Capital Grille,American Restaurant,39.950716,-75.163942,4af2d4cef964a520a9e821e3,Philadelphia,130,american,1
16,Barbuzzo Mediterranean Kitchen & Bar,Mediterranean Restaurant,39.949983,-75.161987,4c7ed709a7958cfa30048f2b,Philadelphia,391,european,1
17,HipCityVeg,Vegetarian / Vegan Restaurant,39.950343,-75.164046,5852d6aad21e030e68cb5512,Philadelphia,26,american,3
19,Oyster House,Seafood Restaurant,39.950415,-75.166533,4a281e64f964a520f4941fe3,Philadelphia,319,american,1
22,Tredici Enoteca,Mediterranean Restaurant,39.949822,-75.162034,565f615b498eeef346958da1,Philadelphia,57,european,3
23,El Vez,Mexican Restaurant,39.949755,-75.161903,40fdb200f964a520380b1fe3,Philadelphia,676,latin,1
24,Beiler's Doughnuts,Donut Shop,39.953697,-75.159307,520f8d6b11d2d0dc2971ac57,Philadelphia,155,None,1


In [43]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset.head()

,name,american,asian,casual,european,latin,Boston,New York,Philadelphia,ranking,likes
1,La Colombe Coffee Roasters,0,0,0,0,0,0,0,1,1,395
4,Fogo De Chão,0,0,0,0,1,0,0,1,1,335
10,Del Frisco's Double Eagle Steak House,0,0,0,0,0,0,0,1,1,275
13,The Capital Grille,1,0,0,0,0,0,0,1,1,130
16,Barbuzzo Mediterranean Kitchen & Bar,0,0,0,1,0,0,0,1,1,391


In [44]:
# Multiple Linear Regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Philadelphia', 
                         'New York', 'Boston']])

y = np.asanyarray(train[['likes']])
regr.fit (x, y)

print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 1.90218899e+01 -1.88476115e+02 -2.62907894e+01 -1.23808389e+02
   8.03447593e+01 -5.87984350e+16 -5.87984350e+16 -5.87984350e+16]]


In [46]:
# Multiple Linear Regression Prediction Capabilities

y_hat= regr.predict(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Philadelphia', 
                         'New York', 'Boston']])

x = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Philadelphia', 
                         'New York', 'Boston']])

y = np.asanyarray(test[['likes']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 43503.22
Variance score: -0.04


In [47]:
# Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Philadelphia', 
                         'New York', 'Boston']])

y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Philadelphia', 
                         'New York', 'Boston']])

y_test = np.asanyarray(test['ranking'])


mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[ 0.42977494 -0.04935696 -0.3229876  -0.44567543 -0.43070241  0.10800761
  0.15220085 -0.26008164]


In [48]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob


# average = None, average = 'micro', average = 'macro', or average = 'weighted'
jaccard_score(y_test, yhat, average='weighted')

0.36605723370429255

In [49]:
log_loss(y_test, yhat_prob)

0.9494344569479657

In [51]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.60      0.90      0.72        20
           2       0.00      0.00      0.00         4
           3       0.43      0.23      0.30        13

    accuracy                           0.57        37
   macro avg       0.34      0.38      0.34        37
weighted avg       0.47      0.57      0.49        37

